In [15]:
#essential libraries
import json
import operator
import shelve
from collections import Counter
from matplotlib import pyplot as plt   
import re
from collections import defaultdict

import numpy as np
import pandas as pd
from scipy.cluster.vq import *
from sklearn.feature_extraction.text import CountVectorizer
import scipy
from sklearn.feature_extraction import DictVectorizer
from scipy.cluster.hierarchy import linkage, dendrogram
import scipy.cluster.hierarchy as hier

In [16]:
building_name = 'ghc'
token_type = 'justseparate'
with open('metadata/%s_sentence_dict_%s.json' % (building_name, token_type), 'r') as fp:
    sentence_dict = json.load(fp)

In [17]:
adder = lambda x,y:x+y
sentence_list = list()
srcid_list = list()
for srcid, sentence in sentence_dict.items():
    srcid_list.append(srcid)
    sentence_list.append(' '.join([word for word in sentence if re.match('[a-zA-Z]+', word)]))

In [18]:
vect = CountVectorizer()
bow = scipy.sparse.coo_matrix(vect.fit_transform(sentence_list))
bow_array = bow.toarray()
feature_set = vect.get_feature_names()
scipy.sparse.hstack

<function scipy.sparse.construct.hstack>

In [19]:
num_of_sensors = len(bow_array)
a = np.array(bow_array[:num_of_sensors])
z = linkage(a, metric='cityblock', method='complete')

In [20]:
#Apply threshold to hierarchical tree to obtain individual clusters. Results stored in equip_map
dists = list(set(z[:,2]))
thresh = (dists[3] + dists[4]) /2 
print("Threshold: ", thresh)
b = hier.fcluster(z,thresh, criterion='distance')

Threshold:  3.5


In [21]:
cluster_dict = defaultdict(list)

for srcid, cluster_id in zip(srcid_list, b):
    cluster_dict[str(cluster_id)].append(srcid)
cluster_dict = dict(cluster_dict)

with open('model/%s_word_clustering_%s.json' % (building_name, token_type.lower()), 'w') as fp:
    json.dump(cluster_dict, fp, indent=4, sort_keys=True)
print(len(cluster_dict))    

761
